In [1]:
import re
import pandas as pd
import jieba
from collections import Counter
from bs4 import BeautifulSoup
import numpy as np

In [2]:
# read .csv to df
df = pd.read_csv('/Users/zhangfurong/Desktop/coding demo/Datacleaned.csv', encoding='utf8')
# delete the col 'source&data'
#df.drop(['source&dates'],axis=1,inplace=True)
# read information of df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 0 to 615
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    616 non-null    int64 
 1   TITLE         616 non-null    object
 2   Abstract      467 non-null    object
 3   source&dates  616 non-null    object
 4   source        616 non-null    object
 5   dates         616 non-null    object
 6   Url           616 non-null    object
dtypes: int64(1), object(6)
memory usage: 33.8+ KB


In [3]:
# read the head
df.head()

,Unnamed: 0,TITLE,Abstract,source&dates,source,dates,Url
0,0,武汉理工研究生自杀 研究生为何会自杀,现在的学生压力很大，如果一旦承受不了，甚至会做出自杀的事情来，近日网曝武汉理工研究生自杀，而...,新淮安网 2018-04-01,新淮安网,2018-04-01,https://www.ringdata.com/news/detail?id=201804...
1,1,美大学生在校内开枪扫射后自杀,胡乱开枪 枪击发生在得克萨斯大学奥斯汀分校。校方发言人朗达·韦尔登说，上午8时许，一名男...,燕赵晚报 2010-09-30,燕赵晚报,2010-09-30,https://www.ringdata.com/news/detail?id=201009...
2,2,女研究生宿舍内自杀,9日，网上各大论坛热传几近相同的帖子，上海海事大学海商法女研究生杨元元在宿舍卫生间自杀身亡，...,滁州网 2009-12-14,滁州网,2009-12-14,https://www.ringdata.com/news/detail?id=200912...
3,3,女研究生跳运河自杀,20日上午8时30分许，北京通州区东关大桥北侧，北京物资学院一名研二女生被发现跳入运河轻生。...,呼和浩特日报 2011-02-22,呼和浩特日报,2011-02-22,https://www.ringdata.com/news/detail?id=201102...
4,4,论文“卡壳”，女研究生自杀,A05-A06版,华声在线网 2011-07-08,华声在线网,2011-07-08,https://www.ringdata.com/news/detail?id=201107...


In [4]:
# define variable: titles(represent the col 'TITLE'), Abstrract(represent the col 'Abstract')
titles = [str(df['TITLE'].tolist())]
abstract = [str(df['Abstract'].tolist())]

In [5]:
# define the function to find out token
def token(string):
    return re.findall('\w+',string)

In [6]:
# Calling 'token(string)' fuunction
title_clean = [''.join(token(str(s))) for s in titles]
abstract_clean =  [''.join(token(str(s))) for s in abstract]
# define array have two lines, one line is title_clean, another one is abstract_clean
inputs = [title_clean, abstract_clean]
#print(inputs)

In [7]:
# load the dictionary that we can defined some particular word, like '网贷'
jieba.load_userdict('/Users/zhangfurong/Desktop/coding demo/stopwords-master/userdict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/7f/pbyrqsk9167dfkx9p9vwj6q00000gn/T/jieba.cache
Loading model cost 0.900 seconds.
Prefix dict has been built successfully.


In [8]:
# define the function to upload the stop_word.txt
def stopwords_list(file_path):
    stopwords = [line.strip() for line in open(file_path, 'r').readlines()]
    return stopwords

In [9]:
# define the cut_word function
def seg_sentence(sentence):
    sentence_seg = jieba.cut(sentence.strip())
    stopwords = stopwords_list('/Users/zhangfurong/Desktop/coding demo/stopwords-master/stopwords_definedbyself.txt')
    out_str = ''
    for word in sentence_seg:
        if word not in stopwords:
            if word != '\t':
                out_str += word
                out_str += " "
    return out_str

In [10]:
# define the function to compute ratio of frequency of one word
def tf(word, count):
    return count[word] / sum(count.values())

In [11]:
if __name__ == '__main__':
    count_list = []
    for line in inputs:
        # WordCut - find out cut-words in titles and abstract and delete
        word_cut = []
        line_seg = seg_sentence(str(line))
        # WordCount
        count = dict(Counter(line_seg.split(" "))) #result of wordcount
        count_list.append(count)
        print(sum(count.values())) 
        # print(count_list) #print the total number dic of wordcount
    # TF 
    for i, count in enumerate(count_list):
        print("Top words in document {}".format(i + 1))
        scores = {word: tf(word, count) for word in count} # Calling mentionabove function to calculate frequency
        sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True) # rank TF
        for word, score in sorted_words[:20]: # print top 20 
            print("\tWord: {},TF: {}".format(word, round(score, 5))) 

4090
13374
Top words in document 1
	Word: 女,TF: 0.04377
	Word: 跳楼自杀,TF: 0.01394
	Word: 疑因,TF: 0.00782
	Word: 自杀身亡,TF: 0.00782
	Word: 警方,TF: 0.00782
	Word: 欲,TF: 0.00685
	Word: 遗书,TF: 0.0066
	Word: 压力,TF: 0.00636
	Word: 遭,TF: 0.00636
	Word: 诈骗,TF: 0.00636
	Word: 网友,TF: 0.00611
	Word: 贷,TF: 0.00587
	Word: 宿舍,TF: 0.00562
	Word: 内,TF: 0.00562
	Word: 身亡,TF: 0.00562
	Word: 烧炭,TF: 0.00562
	Word: 电信,TF: 0.00538
	Word: 一女,TF: 0.00489
	Word: 学费,TF: 0.00489
	Word: 跳海自杀,TF: 0.00465
Top words in document 2
	Word: 女,TF: 0.00636
	Word: 警方,TF: 0.00389
	Word: 身亡,TF: 0.00359
	Word: 女生,TF: 0.00344
	Word: 消息,TF: 0.00314
	Word: 网友,TF: 0.00307
	Word: 近日,TF: 0.00299
	Word: 网,TF: 0.00299
	Word: 说,TF: 0.00292
	Word: 不,TF: 0.00284
	Word: 女儿,TF: 0.00277
	Word: 贷,TF: 0.00277
	Word: 选择,TF: 0.00269
	Word: 调查,TF: 0.00269
	Word: 公安局,TF: 0.00262
	Word: 遗书,TF: 0.00254
	Word: 自杀身亡,TF: 0.00247
	Word: 生命,TF: 0.00247
	Word: 跳楼自杀,TF: 0.00247
	Word: 高校,TF: 0.00239
